# S2. Interoperability

Here, we briefly demonstrate how the single-cell objects may be converted between R and python to run CCC analyses in either language:

In [1]:
library(Seurat, quietly = TRUE)
library(SeuratDisk, quietly = T)
library(zellkonverter, quietly = TRUE)
library(reticulate, quietly = TRUE)
library('Matrix', quietly = T)
anndata<-import('anndata')

data.path <- file.path("..", "..", "data")

Attaching SeuratObject

Seurat v4 was just loaded with SeuratObject v5; disabling v5 assays and
validation routines, and ensuring assays work in strict v3/v4
compatibility mode

Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat

Registered S3 method overwritten by 'zellkonverter':
  method                from      
  py_to_r.numpy.ndarray reticulate



## to Python

For use in Python, we can convert the SingleCellExperiment object generated in [Tutorial 1](./01-Preprocess-Expression.ipynb) to an Anndata object using zellkonverter. See the [companion supplementary python tutorial](../ccc_python/S2_Interoperability.ipynb) for loading these saved files.

In [2]:
# load file
covid_data<-readRDS(file.path(data.path, 'covid_balf_norm.rds'))

# save h5ad
zellkonverter::writeH5AD(covid_data, 
                         X_name = 'logcounts', 
                         file.path(data.path, 'processed_fromR_toPython.h5ad'))

We can also do the same for the Suerat object generated in [Supplementary Tutorial 0](./S0_Preprocess_Expression_Seurat):

In [3]:
# load file
covid_data<-readRDS(file.path(data.path, 'covid_balf_norm_seurat.rds'))

# save as h5ad
file.name<-file.path(data.path, 'processed_fromR_toPython_Seurat.h5Seurat')
suppressMessages({
    SaveH5Seurat(covid_data, filename = file.name)
    Convert(file.name, dest = "h5ad", overwrite = TRUE)
})

## from Python

Here, we can load the expression matrices that were generated in the companion python script using AnnData and saved as h5ad files into Seurat:

Here, we can load the data that was generated in the [python version of Tutorial l](../ccc_python/01-Preprocess-Expression.ipynb) using scanpy into a SCE object:

In [4]:
data_path = file.path("..", "..", "data")
sce_frompython<-zellkonverter::readH5AD(file.path(data_path, 'processed.h5ad'))

We can also do the same into a Seurat object:

In [5]:
adata_to_seurat<-function(adata){
#     raw<-adata$layers[['counts']]$to_adata()
#     raw.counts<-t(as.matrix(raw$X))
    raw.counts<-t(adata$layers[['counts']])
    rownames(raw.counts)<-rownames(adata$var)
    colnames(raw.counts)<-rownames(adata$obs)

    so<-CreateSeuratObject(counts=raw.counts, assay = 'RNA', meta.data = adata$obs)

    norm.counts<-t(as.matrix(adata$X))

    # whether HVGs were filtered for
    if (length(rownames(so)) == dim(norm.counts)[[1]]){
        rownames(norm.counts)<-rownames(so)
    }else{
        rownames(norm.counts)<-rownames(adata$var[adata$var$highly_variable,])
    }
    colnames(norm.counts)<-colnames(so)

    so@assays$RNA@data<-as(norm.counts, "dgCMatrix")
    so@assays$RNA@meta.features<-adata$var
    
    return(so)
}

In [6]:
adata <- anndata$read_h5ad(file.path(data_path, 'processed.h5ad'))
covid_data_fromPython <- adata_to_seurat(adata)

Warning message in asMethod(object):
“sparse->dense coercion: allocating vector of size 11.6 GiB”
